<a href="https://colab.research.google.com/github/Tena-rin/Kintsugi-Splitter/blob/main/Scripts/step3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step3: 3D point cloud generation

## これは何？
このノートは、3D点群化を行うための実装です。

## できること
- 1. 入力画像(input.png)を読み込む
- 2. MoGeを使って3D点群化をする
- 3. plyファイル(o_color.ply)として保存する

## 実行方法
上から順番にセルを実行してください。

In [ ]:
!pip install git+https://github.com/microsoft/MoGe.git
!git clone https://github.com/microsoft/MoGe.git
!pip install -r requirements.txt
!pip install open3d

In [5]:
import cv2
import torch
import open3d as o3d
import numpy as np
from moge.model.v1 import MoGeModel

device = torch.device("cuda")

# Load the model from huggingface hub (or load from local).
model = MoGeModel.from_pretrained("Ruicheng/moge-vitl").to(device)

# Read the input image and convert to tensor (3, H, W) and normalize to [0, 1]
input_image = cv2.cvtColor(cv2.imread("input.png"), cv2.COLOR_BGR2RGB)
input_image = torch.tensor(input_image / 255, dtype=torch.float32, device=device).permute(2, 0, 1)

# Infer
output = model.infer(input_image)
# `output` has keys "points", "depth", "mask" and "intrinsics",
# The maps are in the same size as the input image.
# {
#     "points": (H, W, 3),    # scale-invariant point map in OpenCV camera coordinate system (x right, y down, z forward)
#     "depth": (H, W),        # scale-invariant depth map
#     "mask": (H, W),         # a binary mask for valid pixels.
#     "intrinsics": (3, 3),   # normalized camera intrinsics
# }
# For more usage details, see the `MoGeModel.infer` docstring.

points = output["points"].detach().cpu().numpy()
mask = output["mask"].detach().cpu().numpy() > 0

points_valid = points[mask]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_valid)

points = output["points"].detach().cpu().numpy()
mask = output["mask"].detach().cpu().numpy()

if mask.ndim == 3:
    mask = mask.squeeze(0)

if points.ndim == 4:
    points = points.squeeze(0)

mask = mask > 0

original_image = cv2.cvtColor(cv2.imread("input.png"), cv2.COLOR_BGR2RGB)
original_image = original_image / 255.0

H, W = mask.shape
original_image_resized = cv2.resize(original_image, (W, H))
points_valid = points[mask]
colors_valid = original_image_resized[mask]
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points_valid)
pcd.colors = o3d.utility.Vector3dVector(colors_valid)
o3d.io.write_point_cloud("o_color.ply", pcd, write_ascii=True)

print("色付き点群ファイル o_color.ply を保存しました！")

色付き点群ファイル o_color.ply を保存しました！
